<div class="alert alert-block alert-info">
<h2><center><strong> Participez à la conception d'une voiture autonome :Transformation </strong></center></h2>
        
</div>

<div class="alert alert-block alert-success">  
<strong>1.1 Importing des packages<a id='head-1-2'></a> 📕 📗 📘 📙</strong>
        
</div>

In [15]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.spatial import distance
import os
import _pickle as cPickle
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
# generate random integer values
from random import seed
from random import randint
seed(42)
from scipy.spatial import distance
from scipy import sparse

<div class="alert alert-block alert-danger">  
<h2><center><strong>2.Etude du jeu de données<a id='head-2'></a></strong></center></h2>
        
</div>

<div class="alert alert-block alert-success">  
<strong>2.1 Chargement du jeu de données<a id='head-1-2'></a> 📕 📗 📘 📙</strong>
        
</div>

In [16]:

li = []
for dirname, _, filenames in os.walk('data/clicks'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        df = pd.read_csv(os.path.join(dirname, filename), index_col=None, header=0)
        li.append(df)

dfClicks = pd.concat(li, axis=0, ignore_index=True)
dfClicks.head(10)

,user_id,session_id,session_start,session_size,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,0,1506825423271737,1506825423000,2,157541,1506826828020,4,3,20,1,20,2
1,0,1506825423271737,1506825423000,2,68866,1506826858020,4,3,20,1,20,2
2,1,1506825426267738,1506825426000,2,235840,1506827017951,4,1,17,1,16,2
3,1,1506825426267738,1506825426000,2,96663,1506827047951,4,1,17,1,16,2
4,2,1506825435299739,1506825435000,2,119592,1506827090575,4,1,17,1,24,2
5,2,1506825435299739,1506825435000,2,30970,1506827120575,4,1,17,1,24,2
6,3,1506825442704740,1506825442000,2,236065,1506827536942,4,3,2,1,21,1
7,3,1506825442704740,1506825442000,2,236294,1506827566942,4,3,2,1,21,1
8,4,1506825528135741,1506825528000,2,48915,1506826927593,4,1,17,1,17,1
9,4,1506825528135741,1506825528000,2,44488,1506826957593,4,1,17,1,17,1


In [17]:
users=pd.DataFrame({"user_id":dfClicks['user_id']})
sessions=pd.DataFrame({"session_id":dfClicks['session_id']})

In [18]:
articlesDf = pd.read_csv('data/articles_metadata.csv', index_col=None, header=0)
articlesDf.head()

,article_id,category_id,created_at_ts,publisher_id,words_count
0,0,0,1513144419000,0,168
1,1,1,1405341936000,0,189
2,2,1,1408667706000,0,250
3,3,1,1408468313000,0,230
4,4,1,1407071171000,0,162


In [19]:
with open(r"data/articles_embeddings.pickle", "rb") as input_file:
    e = cPickle.load(input_file)

In [20]:
def getFiveArticles(e, userId):
    
    ee=e
    #get all articles read by user
    var= dfClicks.loc[dfClicks['user_id']==userId]['click_article_id'].tolist()
    #chose randomly one
    value = randint(0, len(var))
    #delete all read articles except the selected one( we do not want to offer user to read something he already read)
    for i in range(0, len(var)):
        if i != value:
            ee=np.delete(ee,[i],0)
    arr=[]
    
    #delecte selected article in the new matrix
    f=np.delete(ee,[value],0)
    #get 5 articles the most similar to the selected one
    for i in range(0,5):
        distances = distance.cdist([ee[value]], f, "cosine")[0]
        min_index = np.argmin(distances)
        f=np.delete(f,[min_index],0)
        #find corresponding matrix in original martix
        result = np.where(e == f[min_index])
        arr.append(result[0][0])
        
    return arr

# Content based filtering based on articles embeddings

#### selecting 5 closest articles using cosin distance

In [21]:
print(getFiveArticles(e, 92059))

[1415, 784, 785, 511, 936]


### Adding words count to article embeddings and selecting 5 closes articles

In [23]:
d=e
max=articlesDf['words_count'].max()
articlesDf['words_count']= articlesDf['words_count'].apply(lambda x: x/max)

d=np.append(d,np.reshape(articlesDf['words_count'].to_numpy(), newshape=(articlesDf['words_count'].shape[0],1)),axis=1)

d.shape

(364047, 251)

In [24]:
print(getFiveArticles(d, 92059))

[173, 219, 144, 9, 202]


In [25]:
n_users = dfClicks['user_id'].values.ravel()
n_users=pd.unique(n_users)
n_sessions = dfClicks['session_id'].values.ravel()
n_sessions=pd.unique(n_sessions)

## DNN Recommender system

Create target variable ( articles embeddings )

In [ ]:
#Target Variable: articles
y= np.zeros((dfClicks.shape[0], 250))

for i in range(0,dfClicks.shape[0]):
    y[i]=e[[dfClicks.iloc[i,: ]['click_article_id']]]

#### Get reordered articles metadata dataframe that corresponds to sessions dataframe

In [ ]:

#processed separately in batches
articles_prepared= pd.DataFrame(columns=["article_id","category_id","created_at_ts","publisher_id","words_count"])

for index, row in dfClicks.iterrows():
    articles_prepared= articles_prepared.append(articlesDf.loc[row['click_article_id']])


#### DNN based on article category, word count, user,ategory_id words_count, user_id, date_created

In [ ]:
n_cats = reordered['category_id'].values.ravel()
n_cats=pd.unique(reordered['category_id'])
n_users = users.values.ravel()
n_users=pd.unique(n_users)

from keras.layers import Input, Embedding, Flatten, Dot, Dense, Multiply, LSTM, Dropout
from keras.models import Model
from keras.optimizers import SGD

user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users.shape[0],250, name="Users-Embedding")(user_input)
user_vec = Dense(250, activation='relu')(user_embedding)

cat_input = Input(shape=[1], name="Category-Input")
cat_embedding = Embedding(reordered['category_id'].max(), 250, name="Catgory-Embedding")(cat_input)
cat_vec = Dense(250, activation='relu')(cat_embedding)

words_input = Input(shape=[1], name="Words-Input")
words_vec=Dense(250, activation='relu')(words_input)

date_input= Input(shape=(1,1),name="Date_Created-Input")
date_vec= LSTM(500, input_shape=(1,1))(date_input)
date_drop=Dropout(0.2)(date_vec)
date_dense= Dense(250, activation="relu")(date_drop)

prod = Multiply()([user_vec,cat_vec,words_vec,date_dense])

dense2= Dense(250, activation ="relu")(prod)
fin= Flatten()(dense2)
model = Model([user_input, cat_input,words_input, date_input], fin)
opt = SGD(learning_rate=0.01)
model.compile(loss='mean_squared_error', optimizer=opt, metrics=['mae', 'mse'] )
model.summary()

In [ ]:
import multiprocessing
_max=reordered['words_count'].max()
model.fit([users,reordered['category_id'],reordered['words_count'].apply(lambda x: x/_max),reordered['created_at_ts'].to_numpy().reshape( -1,1,1)] ,y, epochs=10, batch_size=128,  verbose=1,validation_split=0.2, workers=multiprocessing.cpu_count())

# Collaborative based filtering

memory based (kmea